StepFun 图片理解最佳实践（Jupyter 笔记）

本笔记基于 StepFun 官方文档“图片理解最佳实践”，整理了常用用法与示例代码，方便快速上手与复用。

参考文档：
- [图片理解最佳实践](https://platform.stepfun.com/docs/guide/image_chat)

内容涵盖：
- 简单图片理解（传入单张图片）
- 多轮对话与多图描述策略
- `detail` 参数的使用（low/high）
- 图片优化（resize、压缩）以降低首字延时
- 透明 PNG 适配（RGBA 转 RGB 白底）
- 以 Base64 方式传图
- 常见问题与限制说明

> 目前推荐使用 step-1o-turbo-vision 模型。该模型拥有最强的视频理解能力，推荐开启high detail 模式。


In [2]:
import os
import base64
from openai import OpenAI

# 配置 API 密钥
STEP_API_KEY = os.getenv("STEPFUN_API_KEY")
BASE_URL = "https://api.stepfun.com/v1"

# 初始化客户端
client = OpenAI(api_key=STEP_API_KEY, base_url=BASE_URL)

def recognize_location_from_image(image_path):
    """
    上传图片识别地点信息
    """
    # 将图片转换为 Base64
    def image_to_base64(image_path):
        with open(image_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read())
        return encoded_string.decode('utf-8')
    
    # 转换图片
    image_b64 = image_to_base64(image_path)
    
    # 构造消息
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpg;base64,{image_b64}",
                        "detail": "high"  # 高精度模式，识别更准确
                    }
                },
                {
                    "type": "text",
                    "text": "帮我识别这张图片中的地点信息，包括：具体位置、城市、国家、地标建筑等详细信息"
                }
            ]
        }
    ]
    
    try:
        # 调用视觉理解模型
        completion = client.chat.completions.create(
            model="step-1o-turbo-vision",  # 推荐使用视觉理解模型
            messages=messages,
            max_tokens=500
        )
        
        return completion.choices[0].message.content
        
    except Exception as e:
        return f"识别失败：{str(e)}"

# 使用示例
if __name__ == "__main__":
    image_path = "./media/03_1.jpg"  # 替换为您的图片路径
    result = recognize_location_from_image(image_path)
    print("地点识别结果：")
    print(result)


地点识别结果：
这张图片展示了一些装在白色碗里的蔬菜，摆放在木质桌面上。图片中并没有提供任何关于具体位置、城市、国家或地标建筑等地点信息的线索。图片主要聚焦于食物，背景是普通的木质桌面，没有其他环境或背景信息可以帮助识别具体地点。

### **分析图片内容：**
图片中展示了 **9种不同的蔬菜**，分别放在 **9个白色碗** 里，排列成 **3x3的方阵**。这些蔬菜包括：
1. **左上角**：小土豆
2. **中上**：芦笋
3. **右上角**：红萝卜（可能是樱桃萝卜）
4. **左边中间**：红辣椒
5. **中间**：玉米笋
6. **右边中间**：抱子甘蓝
7. **左下角**：西兰花
8. **中下**：樱桃番茄
9. **右下角**：花菜

### **关于地点信息：**
图片中没有任何可以明确指出具体位置、城市或国家的元素，也没有任何地标建筑或环境特征。图片的右下角有水印，显示 **“©百家号/视觉中国”**，这表明图片可能来自视觉中国的图库，但并不提供关于拍摄地点的信息。

### **结论：**
这张图片是一个**食物展示图**，很可能是在室内拍摄的，背景是木质桌面，没有提供任何关于具体位置、城市、国家或地标建筑的详细信息。如果需要使用这张图片，可能与健康饮食、蔬菜分类或烹饪相关的内容有关。

如果你需要这张图片的来源信息，可以尝试通过水印中的“视觉中国”图库进行搜索，但图片本身不包含任何地点信息。


# 一、实现图片理解
初始化配置

In [22]:
# 环境依赖与客户端初始化
# - 需要 openai>=1.30.0 （API 兼容 StepFun）
# - 需要设置环境变量 API_KEY

import os
from openai import OpenAI

STEPFUN_API_KEY = os.environ.get("STEPFUN_API_KEY")
if not STEPFUN_API_KEY:
    raise RuntimeError("未检测到环境变量 API_KEY，请在运行前设置，例如：export API_KEY='sk-xxx'")

# 指向 StepFun 平台
client = OpenAI(api_key=STEPFUN_API_KEY, base_url="https://api.stepfun.com/v1")

DEFAULT_VISION_MODEL = "step-1o-turbo-vision"

system_prompt = "你是由阶跃星辰提供的AI聊天助手，你除了擅长中文，英文，以及多种其他语言的对话以外，还能够根据用户提供的图片，对内容进行精准的内容文本描述。在保证用户数据安全的前提下，你能对用户的问题和请求，作出快速和精准的回答。同时，你的回答和建议应该拒绝黄赌毒，暴力恐怖主义的内容"




###  1.1简单图片理解

In [13]:
# 简单图片理解示例
# 传入一张图片并请求描述
# 阶跃星辰支持在 image_url 类型中使用 URL 或 Base64 格式的内容，为了保证更好的性能，推荐使用 URL 来完成图片参数的传递

TEST_IMAGE_URL = "https://q0.itc.cn/q_70/images01/20240612/8e0b7aecb4984be19faaa78f4ecd7c92.jpeg"
completion = client.chat.completions.create(
    model=DEFAULT_VISION_MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "用优雅的语言描述这张图片"},
                {
                    "type": "image_url",
                    "image_url": {"url": TEST_IMAGE_URL},
                },
            ],
        },
    ],
)
print("traceid:",completion.id.split('.', 1)[0])
print(completion.choices[0].message.content)
#用优雅的语言描述这张图片

traceid: 30279ed3ae7edd5ace0045e707145dd4
这张图片中，女士身着一袭黑色抹胸礼服，礼服设计简约而优雅，突显出端庄的气质。她的长发自然垂落，微微卷曲，发丝柔顺地披在肩头，散发出一种温婉的美感。她佩戴着精致的珠宝，耳畔是华丽的长款耳环，闪烁着璀璨的光芒，与颈间的钻石项链相得益彰，更添几分高贵与典雅。她的姿态从容，手指上也点缀着一枚戒指，整体造型完美融合了优雅与时尚。背景色调温暖，衬托出她的气质如兰，宛如一幅精心雕琢的艺术作品。


### 1.2 基于图片的多轮对话

In [25]:
TEST_IMAGE_URL = "https://img.meituan.net/leadinimg/fb9411ee59deb2330085f6979aa4a847149136.webp%40watermark%3D0"

messages = []

messages.append({"role": "system", "content": system_prompt})

# 第一轮：带图提问
messages.append({
    "role": "user",
    "content": [
        {"type": "text", "text": "用优雅的语言描述这张图片"},
        {"type": "image_url", "image_url": {"url": TEST_IMAGE_URL}}
    ]
})

# 调一次模型，生成第一轮描述
resp1 = client.chat.completions.create(
    model=DEFAULT_VISION_MODEL,
    messages=messages
)
first_answer = resp1.choices[0].message.content
print( "第一次回答：",first_answer)
# 把模型回答放入上下文，成为对话历史
messages.append({"role": "assistant", "content": first_answer})

# 第二轮：用户继续追问
messages.append({"role": "user", "content": "这张照片中是什么地方？"})

# 再调一次模型，进行多轮对话
resp2 = client.chat.completions.create(
    model=DEFAULT_VISION_MODEL,
    messages=messages
)
second_answer = resp2.choices[0].message.content

print( "第二次回答：",second_answer)


第一次回答： 这张图片展现了一幅令人心醉的巴黎风光。画面中央是雄伟的**埃菲尔铁塔**，铁塔在阳光的照耀下泛着温暖的金色光芒，结构精致而宏伟，彰显出工业革命时期的建筑美学。铁塔上方悬挂着**奥运五环标志**，象征着巴黎即将举办的2024年奥运会，为这座历史建筑增添了一抹现代体育精神的色彩。

铁塔的背景是一片壮丽的**双层彩虹**，横跨天际，从画面的左下角延伸至右上角，色彩斑斓，与灰蓝色的天空形成鲜明对比。彩虹的出现为整个场景增添了一种梦幻与希望的氛围，仿佛是大自然为这座浪漫城市特意绘制的礼物。

在铁塔前方是**塞纳河**，河水波光粼粼，两艘白色游船静静地停泊在河面上，船上写有“Vedettes du Pont de l’Alma”字样，为画面增添了生活气息。河岸两侧绿树成荫，与铁塔和彩虹共同构成了一幅和谐的自然与人文景观。

整个画面在阳光与彩虹的映衬下，充满了浪漫、活力与希望，完美地展现了巴黎作为“光之城”与“浪漫之都”的魅力，同时也预示着巴黎2024年奥运会的辉煌与多彩。

这不仅是一幅风景图，更像是一封写给巴黎的情书，诉说着这座城市的永恒魅力与无限活力。
第二次回答： 这张照片拍摄的是**法国巴黎的埃菲尔铁塔**，背景是著名的**塞纳河**，以及天空中壮丽的**双层彩虹**。画面中埃菲尔铁塔上悬挂的**奥运五环标志**表明，这可能是为**2024年巴黎奥运会**所做的特别装饰，预示着巴黎正在为这场全球体育盛会做准备。

### 具体地点分析：
1. **埃菲尔铁塔（Eiffel Tower）**：
   - 位于巴黎战神广场（Champ de Mars），是巴黎的标志性建筑，也是法国的象征之一。
   - 铁塔建于1889年，为纪念法国大革命100周年及巴黎世博会而建造，如今已成为全球最受欢迎的旅游景点之一。

2. **塞纳河（Seine River）**：
   - 照片前景是塞纳河，河上停泊着游船，这些游船通常用于游客观光，沿着塞纳河欣赏巴黎的美景。
   - 塞纳河贯穿巴黎市中心，两岸分布着许多历史建筑与景点。

3. **奥运五环标志**：
   - 埃菲尔铁塔上悬挂的五环标志表明，巴黎正在为**2024年夏季奥运会**做准备。这一细节突显了巴黎作为奥运会主办城市的特殊时刻。

4. **彩虹**：
   - 天空中出现了**双层彩虹**，这是一

### 1.3 设置detail参数为low和high

- step-1v 默认会选择低分辨率，每张图片 400 token
- step-1o 系列模型低分辨率情况下，默认每张图片 169 token；当 detail 模式为 high 时，图片的 Token 消耗将会基于图片大小进行计算

In [32]:
# 菜单图例
TEST_IMAGE_URL = "https://p0.meituan.net/biztone/7479e268c42b856cbe6dd09969a31b9e2131519.jpg%40watermark%3D0"

def describe_image(img_url: str, detail: str = "high") -> str:
    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "识别图中五常大米的价格"}, #5元一份
                {"type": "image_url", "image_url": {"url": img_url, "detail": detail}},
            ],
        },
    ]
    completion = client.chat.completions.create(model=DEFAULT_VISION_MODEL, messages=messages,temperature= 0)
    # print("traceid:",completion.id.split('.', 1)[0])
    return completion.choices[0].message.content

image_desc_low = describe_image(TEST_IMAGE_URL, detail="low")
print("\nlow detail模式:",image_desc_low,"...\n")
image_desc_high = describe_image(TEST_IMAGE_URL, detail="high")
print("\nhigh detail模式:",image_desc_high, "...\n")



low detail模式: 图中 **五常大米** 的价格是 **15元**。  
\boxed{15元} ...


high detail模式: 图中五常大米的价格是 **5/份**。  
\boxed{5/份} ...



### 1.4 多图图片理解

根据模型不同，一次多轮对话最多可以拥有不超过 10 张照片或 50 张照片
当图片数量可能超过上限（当前单次请求上限为 60 张）时，可先用模型为每张图生成描述，然后将描述作为对话上下文，继续进行真正的问答。

In [ ]:
# 多图理解来识别有几位不同人物
completion = client.chat.completions.create(
    model="step-1o-turbo-vision",
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "这几张图片里出现了几个不同的女生", #3个
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://q0.itc.cn/q_70/images01/20240612/8e0b7aecb4984be19faaa78f4ecd7c92.jpeg" #杨幂
                    },
                },
                {
                    "type":"image_url",
                    "image_url":{
                        "url":"https://gd-hbimg.huaban.com/2b4d0f9fc1515d4d1d2d498b8318dbac3491cc58175ee-83PEcb_fw1200webp"#杨幂
                    }
                },
                {
                    "type":"image_url",
                    "image_url":{
                        "url":"https://gd-hbimg.huaban.com/30576c5c513d28154df3cee1570027bb95a47bc0413d4-6sNdZc_fw1200webp"#杨幂
                    }
                },
                {
                    "type":"image_url",
                    "image_url":{
                        "url":"https://gd-hbimg.huaban.com/14eb4eb2e87b9a46973e9ddca872750043375ad02818c-ildFmT_fw1200webp"#杨幂
                    }
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://img0.baidu.com/it/u=3612072719,173171671&fm=253&app=138&f=JPEG?w=800&h=1067" #李一桐
                    },
                },
                                {
                    "type":"image_url",
                    "image_url":{
                        "url":"https://q6.itc.cn/q_70/images03/20250803/36c739326f074529a145f650f21966a7.jpeg" #李一桐
                    }
                },
                {
                    "type":"image_url",
                    "image_url":{
                        "url":"https://pics3.baidu.com/feed/bf096b63f6246b6003c2629632906143530fa2f8.jpeg@f_auto?token=c549b4f7593f2ecff51c6c1e7fd67201" #白鹿
                    }
                }
            ],
        },
    ],
)
 
print(completion.model_dump_json(indent=3))


{
   "id": "84675650f205d830b148431680898a5d.858cf0596e90686c28e2b955694b46ed",
   "choices": [
      {
         "finish_reason": "stop",
         "index": 0,
         "logprobs": null,
         "message": {
            "content": "这几张图片里出现了 **3个不同的女生**。  \n\n### 判断依据：\n1. **前四张图片（1-4）** 中的女生是同一个人，因为她在外貌、发型和整体气质上一致，尤其是面部轮廓、五官和黑长直的头发特征相似。  \n   - 第1张：黑色礼服，佩戴华丽的耳环和项链。  \n   - 第2张：浅绿色礼服，发型为低盘发，佩戴珍珠耳环。  \n   - 第3张：银色亮片礼服，外搭黑白毛绒外套，项链为金属质感。  \n   - 第4张：深蓝色礼服，搭配浅蓝色薄纱，长卷发披肩。  \n\n   这四张图片应为 **同一个女生**，因为她无论在妆容、五官还是整体气质上都非常一致。  \n\n2. **第5张和第6张图片** 中的女生是 **同一个人**，因为：  \n   - 第5张：她穿着白色衣服，耳朵上戴着金色耳环，表情冷艳，手托下巴，黑长直的发型。  \n   - 第6张：她穿着红色吊带衣服，手里拿着一个装有水的玻璃瓶，表情温柔，头发为深棕色且披散下来。  \n   这两张图片中的女生在外貌、面部特征（如泪痣位置）和整体气质上一致，应为 **同一个人**。  \n\n3. **第7张图片** 中的女生是 **另一个人**，因为她的面部轮廓、五官结构与前两个女生明显不同。她的头发为卷发，妆容更加温柔，且礼服设计与之前的不同，整体感觉更柔和，因此是 **第三个不同的女生**。  \n\n### 结论：\n图片中一共出现了 **3个不同的女生**。  \n- 前四张（1-4）是 **第1个女生**。  \n- 第5张和第6张是 **第2个女生**。  \n- 第7张是 **第3个女生**。",
            "refusal": null,
            "role": "assistant",


### 1.5 使用 Base64 编码来传递图片内容
![image.png](https://platform.stepfun.com/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fbase64image.f0a0bc95.jpg&w=3840&q=75)

In [17]:
# 以 Base64 输入图片
import base64
with open("./media/01_鱼香肉丝.jpg", "rb") as image_file:
    base64_bytes = base64.b64encode(image_file.read())
    base64_bytes = base64_bytes.decode('utf-8')

user_prompt="""
请识别这张图片中的菜名，并给出这道菜的详细做法和所需食材。
"""

completion_b64 = client.chat.completions.create(
    model=DEFAULT_VISION_MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": user_prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{base64_bytes}",
                        "detail": "high",
                    },

                },
            ],
        },
    ],
)

print(completion_b64.id)
print(completion_b64.choices[0].message.content)


731b5f25ed38e3b392defb022b0e7bd3.0516e8f7d7d47949ed6f54d514c91622
这道菜是**鱼香肉丝**，一道经典的川菜，以其咸、甜、酸、辣、香的复合味道而闻名，虽然名字中有“鱼香”，但其实并不含有鱼，而是用调味料模仿出鱼的香味。以下是这道菜的详细做法和所需食材：

---

### **所需食材：**
#### 主料：
- 猪里脊肉：200 克  
#### 配菜：
- 胡萝卜：1 根（中等大小，约 50 克，切丝）  
- 青椒：1 个（约 50 克，切丝）  
- 木耳：10 克（干木耳，提前泡发，切丝）  
- �香菇：2 朵（可选，切片或切丝）  
#### 调料：
- **腌肉调料**：
  - 生抽：1 茶匙  
  - 料酒：1 茶匙  
  - 淀粉：1 茶匙  
  - 食用油：半茶匙  
  - 盐：少许  
- **鱼香汁**：
  - 生抽：1 汤匙  
  - 陈醋：2 汤匙  
  - 白糖：1.5 汤匙  
  - 料酒：1 茶匙  
  - 淀粉：1 茶匙  
  - 清水：3 汤匙  
  - 豆瓣酱：1 茶匙（可选，增加辣味和红油）  
- **其他调料**：
  - 蒜末：适量（约 2-3 瓣）  
  - 姜末：适量（约 1 小块）  
  - 葱花：适量  
  - 红泡椒（或剁椒）：1-2 汤匙（增加辣味和传统鱼香风味，可选）  
  - 食用油：适量  

---

### **详细做法：**

#### **1. 食材准备：**
   - **猪肉处理**：猪里脊肉切成细丝，放入碗中，加入 **1 茶匙生抽**、**1 茶匙料酒**、**1 茶匙淀粉** 和 **少许盐**，抓匀后加入 **半茶匙食用油**，再次抓匀，腌制 10 分钟，让肉丝更嫩。  
   - **配菜处理**：胡萝卜、青椒、木耳、香菇分别洗净，切成细丝备用。如果使用干木耳，需提前用温水泡发，去蒂后切丝。  
   - **鱼香汁调制**：取一个小碗，加入 **1 汤匙生抽**、**2 汤匙陈醋**、**1.5 汤匙白糖**、**1 茶匙料酒**、**1 茶匙淀粉** 和 **3 汤匙清水**，搅拌均匀备用。这一步很关键，鱼香汁的比例决定了味道是否正宗。  

#### **2. 炒制过程：**


## 二、常见问题处理

### 2.1 图片预处理以降低延迟
预处理：对图片缩放与压缩来获得较好的处理速度
- step-1o 对于 detail=low 或默认，可将最长边缩放至 728px
- step-1o 对于 detail=high，可将最长边缩放至 504 的倍数 （示例调整为1008）
- step-1v 对于 detail=low 或默认，可将最长边缩放至 1280px
- step-1v 对于 detail=high，可将最长边缩放至 2688px

In [ ]:

from PIL import Image
# 确保安装了Pillow库
# 运行以下命令以安装Pillow
# pip install Pillow
 
 
# 适用于jpg，压缩图片质量，quality 值通常在 1 到 95 之间，80 是一个常用的平衡点
def compress(input_path, output_path, quality):
    image = Image.open(input_path)
    image.save(output_path, quality=quality,optimize=True)
    print(f"图片已压缩 (Quality: {quality})。")


# 适用于png,转为jpg后压缩图片质量
def compress_png(file_path,output_path, quality):
    image = Image.open(file_path)
    # 注意：如果 PNG 带有透明度，需要先转换为 RGB 模式，否则会报错或产生黑色背景
    if image.mode in ('RGBA', 'P'):
        image = image.convert('RGB')
    image.save(output_path, 'JPEG', quality=quality, optimize=True)
    print(f"PNG 已有损转换为 JPEG (Quality: {quality})。")

# 将图片按照最长边resize，短边等比例缩放，并存储为新图片
def resize_image(input_path, output_path, max_size):
    image = Image.open(input_path)
    width, height = image.size
 
    # 计算新的尺寸
    if width > height:
        new_width = max_size
        new_height = int((max_size / width) * height)
    else:
        new_height = max_size
        new_width = int((max_size / height) * width)
 
    # 调整图片尺寸
    resized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
    resized_image.save(output_path)
    print(f"图片已调整尺寸 (max size: {max_size})。")
 
# 调用函数
resize_image('./media/03_doraemon.png', './media/03_doraemon_resized.png', 1008)
compress_png('./media/03_doraemon.png', './media/03_doraemon_requality.jpg', 80)


图片已调整尺寸 (max size: 1008)。
PNG 已有损转换为 JPEG (Quality: 80)。


### 2.2 透明背景图片
目前 step-1o 系列的模型支持对透明背景的 PNG 图片进行处理，但在使用时，会将透明通道处理景为黑色的情况。你可以通过对图片进行处理，将其背景设置为白色，从而避免模型在推理时，无法正确理解图片中的内容。

In [19]:
# 透明 PNG 适配：RGBA 转 RGB 白色背景
# https://gd-hbimg.huaban.com/7c4795bc026c359c8d17ff19d1d25b574fbb43e03088e9-ena8d7_fw1200webp
from PIL import Image

def convert_rgba_to_rgb_with_white_background(input_path: str, output_path: str) -> None:
    img = Image.open(input_path)
    if img.mode != "RGBA":
        raise ValueError("输入图片不是 RGBA 模式")
    white_background = Image.new("RGB", img.size, (255, 255, 255))
    white_background.paste(img, mask=img.split()[3])
    result = white_background.convert("RGB")
    result.save(output_path)

convert_rgba_to_rgb_with_white_background("./media/03_rgba.webp", "./media/03_rgb.webp")


## 三、其他应用

### 3.1 识别图中的单个主体并图片标注

In [ ]:
import os
import base64
import json
import time
from typing import Dict, Any, Tuple, Optional, List

from PIL import Image, ImageDraw, ImageFont
from openai import OpenAI

STEP_API_KEY = os.environ["STEPFUN_API_KEY"]
BASE_URL="https://api.stepfun.com/v1"

MODEL_NAME = "step-1o-vision-32k"
RESPONSE_FORMAT_JSON = {"type": "json_object"}

# 坐标模式（用于可视化落地到像素）：强制相对坐标（0~1000）
POSITION_MODE = "relative_0_1000"

# 输入/输出
INPUT_IMAGE_PATH = "./media/03_rgb.webp"
OUTPUT_IMAGE_DIR = "./media"     # 目录；程序会自动生成文件名
DRAW_LABEL = "crop"                         # 在图上绘制的框标签
FONT_PATHS = [
    "/System/Library/Fonts/Supplemental/Arial Unicode.ttf",
    "/System/Library/Fonts/Helvetica.ttc",
    "arial.ttf",
]
FONT_SIZE = 20

# 用户自然语言主体描述（示例：请按需修改）
SUBJECT_DESCRIPTION = "识别图中主体，完整包含并保留少量上下文边缘空间"

# 重试
MAX_RETRIES = 5
RETRY_SLEEP_SECONDS = 1

# 调试输出
VERBOSE = True
# ===== 配置区域结束 ======

client = OpenAI(api_key=STEP_API_KEY, base_url=BASE_URL)


def log(*args):
    if VERBOSE:
        print(*args)

# ====== 提示词 =========
def build_system_prompt() -> str:
    """
    严格约束输出为单一裁剪框的 JSON：
    {
      "top_left": [x1, y1],
      "bottom_right": [x2, y2],
      "confidence": 0.95
    }
    """
    return (
        "你是一个专业的图像分析助手。你的任务是：根据用户的自然语言描述，"
        "在给定图片中定位需要裁剪的**核心主体**区域，并仅输出一个裁剪框。\n\n"
        "【输出格式（必须严格遵守）】\n"
        "只输出一个 JSON 对象，不要包含任何其他解释或文字：\n"
        "{\n"
        "  \"top_left\": [x1, y1],\n"
        "  \"bottom_right\": [x2, y2],\n"
        "  \"confidence\": 0.95\n"
        "}\n\n"
        "【坐标与边界规则】\n"
        "1) 使用**相对坐标制**：范围 0~1000 的整数；(0,0) 为左上角，x 向右、y 向下。\n"
        "2) 所有坐标必须位于图像范围内，且满足 x1 < x2、y1 < y2。\n"
        "3) 你的输出只有**一个**裁剪框；如存在多个可能主体，应将它们**合并为一个更大框**以避免重叠问题。\n"
        "4) 框需要**完整包含**用户描述的主体，同时**适度保留上下文**，避免过紧贴边缘。\n"
        "5) 置信度 confidence 取值 0.0–1.0，代表你对本次定位准确度的判断。\n\n"
        "【一致性要求】\n"
        "- 坐标字段名固定为 \"top_left\" 与 \"bottom_right\"；值均为两个整数 [x, y]，范围 0–1000。\n"
        "- 输出中不得包含额外字段、注释或解释。\n"
    )


# ====== 工具函数 =========
def get_image_size(image_path: str) -> Tuple[int, int]:
    with Image.open(image_path) as img:
        return img.size  # (w, h)


def image_to_base64(image_path: str) -> str:
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")


def _clamp(v: int, low: int, high: int) -> int:
    return max(low, min(v, high))


def _to_int2(pt: List[Any]) -> Optional[Tuple[int, int]]:
    if not isinstance(pt, (list, tuple)) or len(pt) != 2:
        return None
    try:
        x = int(round(float(pt[0])))
        y = int(round(float(pt[1])))
        return x, y
    except Exception:
        return None


def validate_relative_bbox(bbox: Dict[str, Any]) -> bool:
    """
    校验相对坐标 0..1000，x1<x2, y1<y2
    """
    tl = _to_int2(bbox.get("top_left"))
    br = _to_int2(bbox.get("bottom_right"))
    if tl is None or br is None:
        return False
    x1, y1 = tl
    x2, y2 = br
    if not (0 <= x1 < x2 <= 1000 and 0 <= y1 < y2 <= 1000):
        return False
    conf = bbox.get("confidence")
    try:
        conf = float(conf)
    except Exception:
        return False
    return 0.0 <= conf <= 1.0


def rel_to_abs_bbox(bbox: Dict[str, Any], img_size: Tuple[int, int]) -> Tuple[int, int, int, int]:
    """
    将相对 0..1000 的坐标转换为像素坐标（用于可视化）
    """
    w, h = img_size
    x1_rel, y1_rel = bbox["top_left"]
    x2_rel, y2_rel = bbox["bottom_right"]
    x1 = int(round(x1_rel / 1000.0 * w))
    y1 = int(round(y1_rel / 1000.0 * h))
    x2 = int(round(x2_rel / 1000.0 * w))
    y2 = int(round(y2_rel / 1000.0 * h))
    # 夹取并保证有效
    x1 = _clamp(x1, 0, w - 2)
    y1 = _clamp(y1, 0, h - 2)
    x2 = _clamp(max(x2, x1 + 1), 1, w - 1)
    y2 = _clamp(max(y2, y1 + 1), 1, h - 1)
    return x1, y1, x2, y2


def _load_font(paths: List[str], size: int) -> ImageFont.FreeTypeFont:
    for p in paths:
        try:
            return ImageFont.truetype(p, size=size)
        except Exception:
            continue
    return ImageFont.load_default()


def annotate_image(image_path: str, bbox_abs: Tuple[int, int, int, int], output_path: str, label: str = "crop") -> None:
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with Image.open(image_path) as img:
        if img.mode != "RGBA":
            img = img.convert("RGBA")

        draw = ImageDraw.Draw(img)
        font = _load_font(FONT_PATHS, FONT_SIZE)

        x1, y1, x2, y2 = bbox_abs
        draw.rectangle([x1, y1, x2, y2], outline=(255, 0, 0, 255), width=2)

        # 居中标签
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        l, t, r, b = draw.textbbox((0, 0), label, font=font)
        tw, th = r - l, b - t
        pad = 6
        rect = [cx - tw // 2 - pad, cy - th // 2 - pad, cx + tw // 2 + pad, cy + th // 2 + pad]
        W, H = img.size
        # 夹取
        dx = -min(0, rect[0]) + min(0, W - rect[2])
        dy = -min(0, rect[1]) + min(0, H - rect[3])
        rect = [rect[0] + dx, rect[1] + dy, rect[2] + dx, rect[3] + dy]

        draw.rectangle(rect, fill=(0, 0, 0, 140))
        draw.text((rect[0] + pad, rect[1] + pad), label, fill=(255, 255, 255, 255), font=font)

        ext = os.path.splitext(output_path)[1].lower()
        if ext in [".jpg", ".jpeg"]:
            background = Image.new("RGB", img.size, (255, 255, 255))
            background.paste(img, mask=img.split()[3])
            background.save(output_path, quality=95)
        else:
            img.save(output_path)

    log(f"标注后的图像已保存至 {output_path}")


# ====== 推理请求 =========
def call_model(image_b64: str, description: str) -> Optional[Dict[str, Any]]:
    sys_prompt = build_system_prompt()
    log("系统提示内容:\n\n", sys_prompt)

    messages = [
        {"role": "system", "content": sys_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"主体描述：{description}"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpg;base64,{image_b64}",
                        "detail": "high",
                    },
                },
                {"type": "text", "text": "请基于以上描述与图像，返回唯一的裁剪框 JSON。"},
            ],
        },
    ]

    try:
        response = client.chat.completions.create(
            messages=messages,
            model=MODEL_NAME,
            response_format=RESPONSE_FORMAT_JSON,
        )
    except Exception as e:
        log(f"API请求失败: {e}")
        return None

    # 打印完整响应（可按需注释）
    log(response)

    try:
        content = response.choices[0].message.content.strip()
        log("原始返回：", content)
        data = json.loads(content)
        return data
    except Exception as e:
        log("解析JSON失败，请检查API返回内容。报错：", e)
        return None

# ========= 主流程 =========
def main():
    img_w, img_h = get_image_size(INPUT_IMAGE_PATH)
    log(f"图像大小: {(img_w, img_h)}")

    image_b64 = image_to_base64(INPUT_IMAGE_PATH)

    attempt = 0
    while attempt < MAX_RETRIES:
        try:
            result = call_model(image_b64, SUBJECT_DESCRIPTION)
            if not result:
                log("未能获取有效的分析结果。")
                attempt += 1
                time.sleep(RETRY_SLEEP_SECONDS)
                continue

            # 校验相对坐标与结构
            if not validate_relative_bbox(result):
                log("返回的坐标不符合 0..1000 或结构不正确，重试...")
                attempt += 1
                time.sleep(RETRY_SLEEP_SECONDS)
                continue

            # 相对 -> 像素（仅用于可视化）
            x1, y1, x2, y2 = rel_to_abs_bbox(result, (img_w, img_h))

            # 生成输出文件名
            os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)
            base_name = os.path.splitext(os.path.basename(INPUT_IMAGE_PATH))[0]
            file_name = f"{base_name}_annotated.png"
            output_path = os.path.join(OUTPUT_IMAGE_DIR, file_name)

            annotate_image(INPUT_IMAGE_PATH, (x1, y1, x2, y2), output_path, DRAW_LABEL)

            # 控制台也打印一次标准 JSON，便于直接消费
            # 注意：此处打印的就是“相对坐标 0..1000”的原始返回
            print(json.dumps({
                "top_left": [int(result["top_left"][0]), int(result["top_left"][1])],
                "bottom_right": [int(result["bottom_right"][0]), int(result["bottom_right"][1])],
                "confidence": float(result["confidence"])
            }, ensure_ascii=False))
            return

        except Exception as e:
            log(f"发生错误: {e}")
            attempt += 1
            time.sleep(RETRY_SLEEP_SECONDS)

    log("达到最大重试次数，操作失败。")


if __name__ == "__main__":
    main()

图像大小: (1200, 2596)
系统提示内容:

 你是一个专业的图像分析助手。你的任务是：根据用户的自然语言描述，在给定图片中定位需要裁剪的**核心主体**区域，并仅输出一个裁剪框。

【输出格式（必须严格遵守）】
只输出一个 JSON 对象，不要包含任何其他解释或文字：
{
  "top_left": [x1, y1],
  "bottom_right": [x2, y2],
  "confidence": 0.95
}

【坐标与边界规则】
1) 使用**相对坐标制**：范围 0~1000 的整数；(0,0) 为左上角，x 向右、y 向下。
2) 所有坐标必须位于图像范围内，且满足 x1 < x2、y1 < y2。
3) 你的输出只有**一个**裁剪框；如存在多个可能主体，应将它们**合并为一个更大框**以避免重叠问题。
4) 框需要**完整包含**用户描述的主体，同时**适度保留上下文**，避免过紧贴边缘。
5) 置信度 confidence 取值 0.0–1.0，代表你对本次定位准确度的判断。

【一致性要求】
- 坐标字段名固定为 "top_left" 与 "bottom_right"；值均为两个整数 [x, y]，范围 0–1000。
- 输出中不得包含额外字段、注释或解释。

ChatCompletion(id='c04b4b8539925c55b40cbfd1b4c2de9c.f5ef13d830b0730a79e3d1601ad9b806', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"top_left": [150, 150], "bottom_right": [850, 950], "confidence": 0.95}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1762851751, model='step-1o-vision-32k', object='chat.completion', ser

### 3.2 识别图片中的多个物体并标注

In [ ]:
import os
import base64
import json
import time
from typing import Dict, Any, Tuple, Optional, List

from PIL import Image, ImageDraw, ImageFont
from openai import OpenAI

STEP_API_KEY = os.environ["STEPFUN_API_KEY"]
BASE_URL="https://api.stepfun.com/v1"

MODEL_NAME = "step-1o-vision-32k"
RESPONSE_FORMAT_JSON = {"type": "json_object"}

# 坐标模式（用于可视化落地到像素）：强制相对坐标（0~1000）
POSITION_MODE = "relative_0_1000"

# 输入/输出
INPUT_IMAGE_PATH = "./media/03_doraemon_requality.jpg"
OUTPUT_IMAGE_PATH = "./media/03_doraemon_requality_annotated.png"  

DRAW_LABEL = "crop"                         # 在图上绘制的框标签
FONT_PATHS = [
    "/System/Library/Fonts/Supplemental/Arial Unicode.ttf",
    "/System/Library/Fonts/Helvetica.ttc",
    "arial.ttf",
]
FONT_SIZE = 20

# 用户自然语言主体描述（示例：请按需修改）
SUBJECT_DESCRIPTION = "识别图中的卡通角色面部，注意要完整包含并保留少量边缘空间"

# 调试输出
VERBOSE = True


client = OpenAI(api_key=STEP_API_KEY, base_url=BASE_URL)


def log(*args):
    if VERBOSE:
        print(*args)


# ===== 提示词（Prompt） ===
def build_system_prompt() -> str:
 
    return (
        """
        你是专业图像分析助手，需精准分析用户提供的图片，根据其描述识别所有指定主体区域，为每个主体输出准确包含主体的边界框坐标。

        ### 输出格式（严格遵循）：
        仅输出JSON格式的字符串数组，每个元素为"x1,y1,x2,y2"（左上角x1,y1，右下角x2,y2），无任何额外文字。示例：
        ["ax1,ay1,ax2,ay2","bx1,by1,bx2,by2"]

        ### 坐标规则：
        1. 坐标系：以图片左上角为原点(0,0)，x轴向右递增，y轴向下递增
        2. 数值标准：采用相对坐标（0-1000整数），0为最左/最上，1000为最右/最下（需符合图片比例）
        3. 有效性：确保x1 < x2、y1 < y2，且所有坐标在0-1000范围内

        ### 核心要求：
        - **比例匹配**：边界框的宽高比（x2-x1 : y2-y1）必须接近主体本身的自然宽高比
        - **中心一致**：物体的中心和边界框的中心应尽量重合，
        - **边框接近**：框的上、下、左、右边缘均需**紧挨**主体的自然轮廓，刚好把主体框住，与主体边缘贴合没有缝隙

        ### 错误规避：
        - 边缘离主体过远，导致框内包含大量空白
        - 边缘切割主体，导致主体部分缺失
        """
 )
   
# ====== 工具函数 =========
def get_image_size(image_path: str) -> Tuple[int, int]:
    with Image.open(image_path) as img:
        return img.size  # (w, h)


def image_to_base64(image_path: str) -> str:
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")


def _clamp(v: int, low: int, high: int) -> int:
    return max(low, min(v, high))


def _to_int2(pt: List[Any]) -> Optional[Tuple[int, int]]:
    if not isinstance(pt, (list, tuple)) or len(pt) != 2:
        return None
    try:
        x = int(round(float(pt[0])))
        y = int(round(float(pt[1])))
        return x, y
    except Exception:
        return None


def validate_relative_bbox(bbox: Dict[str, Any]) -> bool:
    """
    校验相对坐标 0..1000，x1<x2, y1<y2
    """
    tl = _to_int2(bbox.get("top_left"))
    br = _to_int2(bbox.get("bottom_right"))
    if tl is None or br is None:
        return False
    x1, y1 = tl
    x2, y2 = br
    if not (0 <= x1 < x2 <= 1000 and 0 <= y1 < y2 <= 1000):
        return False
    conf = bbox.get("confidence")
    try:
        conf = float(conf)
    except Exception:
        return False
    return 0.0 <= conf <= 1.0


def rel_to_abs_bbox(bbox: Dict[str, Any], img_size: Tuple[int, int]) -> Tuple[int, int, int, int]:
    """
    将相对 0..1000 的坐标转换为像素坐标（用于可视化）
    """
    w, h = img_size
    x1_rel, y1_rel = bbox["top_left"]
    x2_rel, y2_rel = bbox["bottom_right"]
    x1 = int(round(x1_rel / 1000.0 * w))
    y1 = int(round(y1_rel / 1000.0 * h))
    x2 = int(round(x2_rel / 1000.0 * w))
    y2 = int(round(y2_rel / 1000.0 * h))
    # 夹取并保证有效
    x1 = _clamp(x1, 0, w - 2)
    y1 = _clamp(y1, 0, h - 2)
    x2 = _clamp(max(x2, x1 + 1), 1, w - 1)
    y2 = _clamp(max(y2, y1 + 1), 1, h - 1)
    return x1, y1, x2, y2


def _load_font(paths: List[str], size: int) -> ImageFont.FreeTypeFont:
    for p in paths:
        try:
            return ImageFont.truetype(p, size=size)
        except Exception:
            continue
    return ImageFont.load_default()


class ImageAnnotator:
    def __init__(self, image_path: str):
        """初始化标注器，加载图片并准备绘制对象"""
        # 只加载一次图片
        self.image_path = image_path
        self.img = Image.open(image_path)
        # 转换为RGBA以支持透明图层
        if self.img.mode != "RGBA":
            self.img = self.img.convert("RGBA")
        # 创建绘制对象
        self.draw = ImageDraw.Draw(self.img)
        # 加载字体
        self.font = _load_font(FONT_PATHS, FONT_SIZE)
        print("字体加载:",self.font)


    def add_annotation(self, bbox_abs: Tuple[int, int, int, int], label: str = "crop"):
        """添加一个标注（可多次调用）"""
        x1, y1, x2, y2 = bbox_abs
        
        # 绘制边界框
        self.draw.rectangle([x1, y1, x2, y2], outline=(255, 0, 0, 255), width=2)
        
        # 绘制居中标签
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        l, t, r, b = self.draw.textbbox((0, 0), label, font=self.font)
        tw, th = r - l, b - t
        pad = 6
        rect = [
            cx - tw // 2 - pad, 
            cy - th // 2 - pad, 
            cx + tw // 2 + pad, 
            cy + th // 2 + pad
        ]
        W, H = self.img.size
        
        # 调整标签位置避免超出图像边界
        dx = -min(0, rect[0]) + min(0, W - rect[2])
        dy = -min(0, rect[1]) + min(0, H - rect[3])
        rect = [rect[0] + dx, rect[1] + dy, rect[2] + dx, rect[3] + dy]
        
        # 绘制标签背景和文字
        self.draw.rectangle(rect, fill=(0, 0, 0, 140))
        self.draw.text(
            (rect[0] + pad, rect[1] + pad), 
            label, 
            fill=(255, 255, 255, 255), 
            font=self.font
        )

    def save(self, output_path: str):
        """保存标注后的图片"""
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        ext = os.path.splitext(output_path)[1].lower()
        
        # 处理JPG格式的透明背景问题
        if ext in [".jpg", ".jpeg"]:
            background = Image.new("RGB", self.img.size, (255, 255, 255))
            background.paste(self.img, mask=self.img.split()[3])  # 使用alpha通道作为掩码
            background.save(output_path, quality=95)
        else:
            self.img.save(output_path)
        
        # 关闭图片（释放资源）
        self.img.close()
        print(f"标注后的图像已保存至 {output_path}")


def call_model(image_b64: str, description: str) -> Optional[Dict[str, Any]]:
    sys_prompt = build_system_prompt()

    messages = [
        {"role": "system", "content": sys_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"主体描述：{description}"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpg;base64,{image_b64}",
                        "detail": "high",
                    },
                },
                {"type": "text", "text": "请基于以上描述与图像，返回所有符合要求的裁剪框数组。"},
            ],
            "temprature": 0.0,
        },
    ]

    try:
        response = client.chat.completions.create(
            messages=messages,
            model=MODEL_NAME,
            response_format=RESPONSE_FORMAT_JSON,
        )
    except Exception as e:
        log(f"API请求失败: {e}")
        return None

    try:
        content = response.choices[0].message.content.strip()
        log("原始返回：", content)
        data = json.loads(content)
        return data
    except Exception as e:
        log("解析JSON失败，请检查API返回内容。报错：", e)
        return None


def main():
    img_w, img_h = get_image_size(INPUT_IMAGE_PATH)
    log(f"图像大小: {(img_w, img_h)}")

    image_b64 = image_to_base64(INPUT_IMAGE_PATH)

    result = call_model(image_b64, SUBJECT_DESCRIPTION)
    if not result:
        log("未能获取有效的分析结果。")
    
    annotator = ImageAnnotator(INPUT_IMAGE_PATH)
    print(result)

    for i, item in enumerate(result, 1):
        # 关键修复：字符串按逗号分割，转换为整数元组
        x1_rel, y1_rel, x2_rel, y2_rel = map(float, item.split(','))
    
        # 步骤2：将相对坐标转换为像素坐标（乘以图片宽高）
        x1 = int(x1_rel * img_w/1000.0)  # x方向坐标 × 宽度
        y1 = int(y1_rel * img_h/1000.0)  # y方向坐标 × 高度
        x2 = int(x2_rel * img_w/1000.0)
        y2 = int(y2_rel * img_h/1000.0)
        
        # 转换为元组（像素坐标）
        bbox_abs = (x1, y1, x2, y2)
        # 传递给add_annotation
        annotator.add_annotation(bbox_abs, label=f"识别_{i}")
    
    # 3. 最终保存图片
    annotator.save(OUTPUT_IMAGE_PATH)


if __name__ == "__main__":
    main()

图像大小: (2000, 2664)
ChatCompletion(id='e34ba07123a62c86bf36e5b2d333ccd7.fc18784054785e66df59e81e04187877', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='["144,388,348,536","212,236,504,450","524,288,722,438","670,370,910,527","346,476,660,712"]', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1762855480, model='step-1o-vision-32k', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=82, prompt_tokens=2607, total_tokens=2689, completion_tokens_details=None, prompt_tokens_details=None, cached_tokens=256))
原始返回： ["144,388,348,536","212,236,504,450","524,288,722,438","670,370,910,527","346,476,660,712"]
字体加载: <PIL.ImageFont.FreeTypeFont object at 0x110a22d50>
['144,388,348,536', '212,236,504,450', '524,288,722,438', '670,370,910,527', '346,476,660,712']
标注后的图像已保存至 ./media/03_doraemon_requality_annotated.png
